In [18]:
import cv2
import numpy as np
import mediapipe as mp
import joblib
import os
import math

# ==============================================================================
# --- 1. KONFIGURASI ---
# ==============================================================================

MODEL_DIR = 'model'

# Tentukan threshold dalam DERAJAT agar mudah dipahami
PITCH_THRESHOLD_DEG = 30
HEAD_DOWN_FRAMES_THRESHOLD = 20

# ==============================================================================
# --- 2. FUNGSI BANTUAN (TIDAK BERUBAH) ---
# ==============================================================================

def create_feature_vector(face_landmarks):
    # ... (Fungsi ini tidak perlu diubah, salin dari sebelumnya)
    anchor_point = face_landmarks.landmark[1]
    p_left = face_landmarks.landmark[359]
    p_right = face_landmarks.landmark[130]
    scale_distance = np.linalg.norm([p_left.x - p_right.x, p_left.y - p_right.y])
    if scale_distance < 1e-6: return None
    feature_vector = []
    for i in range(len(face_landmarks.landmark)):
        if i == 1: continue
        landmark = face_landmarks.landmark[i]
        feature_vector.extend([(landmark.x - anchor_point.x) / scale_distance, (landmark.y - anchor_point.y) / scale_distance, (landmark.z - anchor_point.z) / scale_distance])
    return np.array(feature_vector)

def draw_axes(img, pitch, yaw, roll, nose_2d, size=100):
    # ... (Fungsi ini tidak perlu diubah, salin dari sebelumnya)
    # Fungsi ini sudah menerima input derajat, jadi kita akan berikan nilai yang sudah dikonversi
    pitch = pitch * np.pi / 180
    yaw = -(yaw * np.pi / 180)
    roll = roll * np.pi / 180
    Rx = np.array([[1, 0, 0], [0, math.cos(pitch), -math.sin(pitch)], [0, math.sin(pitch), math.cos(pitch)]])
    Ry = np.array([[math.cos(yaw), 0, math.sin(yaw)], [0, 1, 0], [-math.sin(yaw), 0, math.cos(yaw)]])
    Rz = np.array([[math.cos(roll), -math.sin(roll), 0], [math.sin(roll), math.cos(roll), 0], [0, 0, 1]])
    R = Rz @ Ry @ Rx
    axis = np.array([[size, 0, 0], [0, size, 0], [0, 0, size]])
    rotated_axis = R @ axis
    p1 = (int(nose_2d[0]), int(nose_2d[1]))
    p2_pitch = (int(nose_2d[0] + rotated_axis[0, 1]), int(nose_2d[1] + rotated_axis[1, 1])); cv2.line(img, p1, p2_pitch, (0, 255, 0), 3)
    p2_yaw = (int(nose_2d[0] + rotated_axis[0, 0]), int(nose_2d[1] + rotated_axis[1, 0])); cv2.line(img, p1, p2_yaw, (255, 0, 0), 3)
    p2_roll = (int(nose_2d[0] + rotated_axis[0, 2]), int(nose_2d[1] + rotated_axis[1, 2])); cv2.line(img, p1, p2_roll, (0, 0, 255), 3)
    return img

# ==============================================================================
# --- 3. EKSEKUSI UTAMA (DENGAN LOGIKA RADIAN) ---
# ==============================================================================

def main():
    try:
        models = { 'pitch': joblib.load(os.path.join(MODEL_DIR, 'xgb_pitch_model.joblib')), 'yaw': joblib.load(os.path.join(MODEL_DIR, 'xgb_yaw_model.joblib')), 'roll': joblib.load(os.path.join(MODEL_DIR, 'xgb_roll_model.joblib')) }
        print("✅ Model head pose berhasil dimuat.")
    except FileNotFoundError:
        print(f"❌ Error: File model tidak ditemukan. Pastikan path '{MODEL_DIR}' sudah benar.")
        return

    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    cap = cv2.VideoCapture(0)
    print("🎥 Webcam berhasil dibuka. Tekan 'q' untuk keluar.")

    head_down_counter = 0
    
    # --- PERUBAHAN 1: KONVERSI THRESHOLD KE RADIAN ---
    # Lakukan ini sekali saja sebelum loop untuk efisiensi
    PITCH_THRESHOLD_RAD = PITCH_THRESHOLD_DEG * np.pi / 180

    while cap.isOpened():
        success, frame = cap.read()
        if not success: break
        
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)
        drowsiness_alert = False

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                feature_vector = create_feature_vector(face_landmarks)
                
                if feature_vector is not None:
                    feature_vector = feature_vector.reshape(1, -1)
                    
                    # Prediksi dari model sekarang dalam satuan RADIAN
                    pitch_rad = models['pitch'].predict(feature_vector)[0]
                    yaw_rad = models['yaw'].predict(feature_vector)[0]
                    roll_rad = models['roll'].predict(feature_vector)[0]
                    
                    # --- PERUBAHAN 2: LOGIKA KANTUK MENGGUNAKAN RADIAN ---
                    # Kita asumsikan menunduk adalah nilai pitch positif
                    if pitch_rad < -PITCH_THRESHOLD_RAD:
                        head_down_counter += 1
                    else:
                        head_down_counter = 0
                    
                    if head_down_counter >= HEAD_DOWN_FRAMES_THRESHOLD:
                        drowsiness_alert = True
                    
                    # --- PERUBAHAN 3: KONVERSI KEMBALI KE DERAJAT UNTUK TAMPILAN ---
                    pitch_deg = pitch_rad * 180 / np.pi
                    yaw_deg = yaw_rad * 180 / np.pi
                    roll_deg = roll_rad * 180 / np.pi
                    
                    # Visualisasi (menggunakan nilai derajat)
                    nose_2d = (face_landmarks.landmark[1].x * frame.shape[1], face_landmarks.landmark[1].y * frame.shape[0])
                    frame = draw_axes(frame, pitch_deg, yaw_deg, roll_deg, nose_2d)
                    cv2.putText(frame, f"Pitch: {pitch_deg:.2f}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
                    cv2.putText(frame, f"Yaw: {yaw_deg:.2f}", (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
                    cv2.putText(frame, f"Roll: {roll_deg:.2f}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

        if drowsiness_alert:
            cv2.putText(frame, "!!! PERINGATAN KANTUK !!!", (100, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3, cv2.LINE_AA)
            cv2.putText(frame, "Kepala Terkulai", (190, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        cv2.imshow('Real-time Drowsiness Detection', frame)
        if cv2.waitKey(5) & 0xFF == ord('q'): break

    cap.release()
    cv2.destroyAllWindows()
    face_mesh.close()
    print("Aplikasi ditutup.")

if __name__ == '__main__':
    main()

✅ Model head pose berhasil dimuat.
🎥 Webcam berhasil dibuka. Tekan 'q' untuk keluar.
Aplikasi ditutup.
